In [2]:
import os
import json
from dotenv import load_dotenv
load_dotenv('/Users/joe.constantino/Desktop/playground/tableau_langchain/.env')

#base langchain library imports
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType, create_tool_calling_agent, AgentExecutor

#tableau_langchain imports
from readMetadata import read
from setPrompt import instantiate_prompt, get_values
from getDataTool import get_data

In [3]:
#read in the environment varibales to query a Tableau Published Data Source from a target Tableau Cloud Site or Server Instance
datasource = os.getenv('DATASOURCE_LUID')
read_metadata_url = os.getenv('READ_METADATA')
query_datasource = os.getenv('QUERY_DATASOURCE')
token = os.getenv('AUTH_TOKEN')

In [4]:
## Use the read module to pull in the necessary metadata from your target Tableau Published Data Source. This will module also augment the
## base metadata provided by the read_metadata endpoint by removing unneccesary fields and adding sample field values to each string field.
metadata = read(read_url=read_metadata_url, datasource_luid=datasource, auth_secret=token)

# use the instaniate_prompt module to add the data model metadata to the nlq_to_vds prompt
prompt = instantiate_prompt(metadata = metadata)

reading in your field metadata...
looking up field values...
prompt is ready!


In [5]:
# Initialize an LLM and an agent with the get_data tool to answer natural language questions with Tableau Published Datasources
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
tools = [get_data]

tableauHeadlessAgent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,  # Use OpenAI's function calling
    verbose=True
)

# Run the agent
response = tableauHeadlessAgent.run("what are sales per region?")
print(response)

/var/folders/3t/f2z2c6_n3951jmt86v5rvgp80000gp/T/ipykernel_54741/1583175394.py:5: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  tableauHeadlessAgent = initialize_agent(
/var/folders/3t/f2z2c6_n3951jmt86v5rvgp80000gp/T/ipykernel_54741/1583175394.py:13: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = tableauHeadlessAgent.run("what are sales per region?")




> Entering new AgentExecutor chain...

Invoking: `get_data` with `{'query': 'sales per region'}`


[{'Region': 'West', 'SUM(Sales)': 739813.61}, {'Region': 'East', 'SUM(Sales)': 691828.17}, {'Region': 'Central', 'SUM(Sales)': 503170.67}, {'Region': 'South', 'SUM(Sales)': 391721.91}]The sales per region are as follows:

- **West**: $739,813.61
- **East**: $691,828.17
- **Central**: $503,170.67
- **South**: $391,721.91

> Finished chain.
The sales per region are as follows:

- **West**: $739,813.61
- **East**: $691,828.17
- **Central**: $503,170.67
- **South**: $391,721.91


End here

The cells below are experimental cells to run the agent with a model running locally using ollama

In [4]:
%pip install -U langchain-ollama

Note: you may need to restart the kernel to use updated packages.


In [14]:
from langchain_ollama.llms import OllamaLLM
local_llm = OllamaLLM(model="llama3.1")


In [ ]:
tools = [get_data]
tableauHeadlessAgent = initialize_agent(
    tools,
    local_llm,
    agent=AgentType.OPENAI_FUNCTIONS,  # Use OpenAI's function calling
    verbose=True
)

# Run the agent
response = tableauHeadlessAgent.run("what are sales per segment?")
print(response)

In [22]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to answer the input question by using get_data tool to retrieve data from a tableau published data source.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)
# Construct the Tools agent
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke("what are sales per segment?")

IndexError: list index out of range